# A Guided Tour of Ray Core: Ray Actor Tree Pattern

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_05_multiprocess_pool.ipynb) <br>
⬅️ [Previous notebook](./ex_03_remote_classes.ipynb) <br>


### Learning objectives
In this this tutorial, we revisit Ray Actors and learn more about:
 * Common Ray Actors patterns used in Ray native libraries for writing distributed Actors
 * How to pass Ray Actors to remote tasks for distributed computing

Let's implement a simple example to illustrate this pattern.

# Tree of Actors Pattern

A common pattern used in Ray libraries [Ray Tune](https://docs.ray.io/en/latest/tune/index.html), [Ray Train](https://docs.ray.io/en/latest/train/train.html), and [RLlib](https://docs.ray.io/en/latest/rllib/index.html) to train models in a parallel or conduct distributed HPO.

In this common pattern, tree of actors, a collection of workers as actors, are managed by a supervisor actor. For example, you want to train multiple models, each of a different type, at the same time, while being able to inspect its state during its training.

<img src="https://docs.ray.io/en/latest/_images/tree-of-actors.svg" width="50%" height="30%">


---

First, let's start Ray…

In [1]:
import logging
import time
import ray
import random
from random import randint
import numpy as np

In [2]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

RayContext(dashboard_url='127.0.0.1:8265', python_version='3.8.13', ray_version='2.0.0rc1', ray_commit='321d8717f73995153d4f9abe98678160831090e1', address_info={'node_ip_address': '127.0.0.1', 'raylet_ip_address': '127.0.0.1', 'redis_address': None, 'object_store_address': '/tmp/ray/session_2022-08-17_16-20-12_249826_34957/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-08-17_16-20-12_249826_34957/sockets/raylet', 'webui_url': '127.0.0.1:8265', 'session_dir': '/tmp/ray/session_2022-08-17_16-20-12_249826_34957', 'metrics_export_port': 56070, 'gcs_address': '127.0.0.1:57447', 'address': '127.0.0.1:57447', 'dashboard_agent_listen_port': 52365, 'node_id': 'a7e32edf31a38a17b9eb97b36473d00bd074134a46e44e9a4e30f184'})

## Example 1: Supervisor and worker actor pattern

### Generic model factory utility  

This factory generates a few specify type of models (they are fake 😏): regression, classification, or neural network, and will have its respective training function. Each model will be in a particular state  during training. The final state is `DONE`.

In [ ]:
# Factory function to return an instance of a model type
def model_factory(m: str, func: object):
    return Model(m, func)

# states to inspect or checkpoint
STATES = ["RUNNING", "PENDING", "DONE"]

class Model:

    def __init__(self, m:str, func: object):
        self._model = m
        self._func = func

    def train(self):
        # do some training work here for the respective model type
        self._func()

### Create a Worker Actor
This worker actor will train each model. When the model's state reaches `DONE`, we stop training

In [8]:
@ray.remote
class Worker(object):
    def __init__(self, m:str, func: object):
        # type of a model: lr, cl, or nn
        self._model = m  
        self._func = func
       
    # inspect its current state and return it. For now
    # it could be in one of the states
    def state(self) -> str:
        return random.choice(STATES)
    
    # Create the model from the factory for this worker and 
    # do the training by invoking its respective objective function 
    # for that model
    def work(self) -> None:
        model_factory(self._model, self._func).train()

### Create Supervisor Actor 
The supervisor creates three actors, each with its own respective training model type and its training function.

In [9]:
# Define respective model training functions

def lf_func():
    # do some training work for linear regression
    time.sleep(1)
    return 0

def cl_func():
     # do some training work for classification
    time.sleep(1)
    return 0

def nn_func():
     # do some training work for neural networks
    time.sleep(1)
    return 0

@ray.remote
class Supervisor:
    def __init__(self):
        # Create three Actor Workers, each by its unique model type and 
        # their respective training function
        self.workers = [Worker.remote(name, func) for (name, func) 
                        in [("lr", lf_func), ("cl",cl_func), ("nn", nn_func)]]
                        
    def work(self):
        # do the work 
        [worker.work.remote() for worker in self.workers]
        
    def terminate(self):
        [ray.kill(worker) for worker in self.workers]
        
    def state(self):
        return ray.get([worker.state.remote() for worker in self.workers])

### Create a Actor instance for supervisor and launch its workers

In [10]:
sup = Supervisor.remote()

# Launch remote actors as workers
sup.work.remote()

ObjectRef(32d950ec0ccf9d2a7535cf0794422421c201b26d0100000001000000)

### Look at the Ray Dashboard

You should see Actors running as process on the workders nodes
 * Supervisor
 * Workers
 
Also, click on the `Logical View` to view more metrics and data on individual Ray Actors

In [11]:
# check their status
while True:
    # Fetch the states of all its workers
    states = ray.get(sup.state.remote())
    print(states)
    # check if all are DONE
    result = all('DONE' == e for e in states)
    if result:
        # Note: Actor processes will be terminated automatically when the initial actor handle goes out of scope in Python. 
        # If we create an actor with actor_handle = ActorClass.remote(), then when actor_handle goes out of scope and is destroyed, 
        # the actor process will be terminated. Note that this only applies to the original actor handle created for the actor 
        # and not to subsequent actor handles created by passing the actor handle to other tasks.
        
        # kill supervisors' all workers manually, only for illustrtation and demo
        sup.terminate.remote()

        # kill the supervisor manually, only for illustration and demo
        ray.kill(sup)
        break

['RUNNING', 'RUNNING', 'DONE']
['PENDING', 'PENDING', 'RUNNING']
['DONE', 'PENDING', 'DONE']
['RUNNING', 'DONE', 'RUNNING']
['PENDING', 'PENDING', 'RUNNING']
['RUNNING', 'DONE', 'PENDING']
['DONE', 'PENDING', 'DONE']
['DONE', 'DONE', 'DONE']


### Recap

What we have demonstrated above is an Actor tree design pattern, commonly used in Ray for writing distributed applications. In particular, Ray's native libraries such as Train, Tune, Serve, and RLib and [Ray AIR's]() components use it for distributed training and tuning trials.

### Any questions?

Let's look at another example in a similar tree of actors (and tasks) pattern

## Example 2: Passing Actor handles to Ray Tasks

Actors are versatile: they can instantiated and passed to remote Ray tasks or even other actors. 

Let's consider writing a distributed messaging service, where workers may post messages to update the state of the messaging service. This could be a logging or monitoring service. For example, [WhyLabs](https://www.anyscale.com/blog/running-and-monitoring-distributed-ml-with-ray-and-whylogs) implemented a variation of this usage pattern to monitor Ray Serve deployments. Since tasks and actors are accessiible as Python
objects, they can be passed around to other Python classes or functions.

You can pass actor handle instances to remote Ray tasks, which can change the actor's 
state. The `MessageActor` keeps or clears messages, depending on the method
invoked.

In [12]:
@ray.remote
class MessageActor(object):
    def __init__(self):
        # Keep the state of all the messages received
        self.messages = []
    
    def add_message(self, message):
        self.messages.append(message)
    
    # reset and clear all messages
    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages

Create a worker doing some specific work, such as updating a DB or posting a tweet or
checking a status of a process and then sends a message to the actor.

**NOTE**: _Question: What does this remind of you from the previous lessons?_ 

(PS)

In [14]:
@ray.remote
def worker(message_actor, j):
    for i in range(10):
        time.sleep(1)
        message_actor.add_message.remote(
            f"Message {i} from worker {j}.")

In [15]:
message_actor = MessageActor.remote()

Start three worker tasks that update the `MessageActor` service since each Ray task gets the handle to the `MessageActor`

In [16]:
[worker.remote(message_actor, j) for j in range(3)]

[ObjectRef(2751d69548dba956ffffffffffffffffffffffff0100000001000000),
 ObjectRef(71b133a11e1c461cffffffffffffffffffffffff0100000001000000),
 ObjectRef(5d4b8d1788f12d2dffffffffffffffffffffffff0100000001000000)]

Since the workers are already launched, let's get actor's state.

In [17]:
for _ in range(10):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New messages\n:", new_messages)
    time.sleep(1)

New messages
: ['Message 0 from worker 0.', 'Message 0 from worker 2.', 'Message 0 from worker 1.', 'Message 1 from worker 0.', 'Message 1 from worker 2.', 'Message 1 from worker 1.', 'Message 2 from worker 0.', 'Message 2 from worker 2.', 'Message 2 from worker 1.']
New messages
: ['Message 3 from worker 0.', 'Message 3 from worker 2.', 'Message 3 from worker 1.']
New messages
: ['Message 4 from worker 0.', 'Message 4 from worker 2.', 'Message 4 from worker 1.']
New messages
: ['Message 5 from worker 2.', 'Message 5 from worker 1.', 'Message 5 from worker 0.']
New messages
: ['Message 6 from worker 0.', 'Message 6 from worker 2.', 'Message 6 from worker 1.']
New messages
: ['Message 7 from worker 2.', 'Message 7 from worker 1.', 'Message 7 from worker 0.']
New messages
: ['Message 8 from worker 0.', 'Message 8 from worker 2.', 'Message 8 from worker 1.']
New messages
: ['Message 9 from worker 0.', 'Message 9 from worker 1.', 'Message 9 from worker 2.']
New messages
: []
New messages
:

In [19]:
ray.shutdown()

### Exercises

1. Add a remote class, such as a logging actor, that keeps states by logging info (may be only in memory) about N experiments, where (N <=3).
2. Implement actor methods that alters the state. That is, it tracks results of 9 separate runs per each experiment.
3. Write a separate Ray task that executes 9 runs per each experiment.
4. Instantiate the actor and call its methods from within the remote Ray task.

### Solution hints

This solution is just a structural hint. There are few missing bits:
 * instantiation of `LoggingActor`
 * Need to use `ray.get()` to fetch the values from the object store

In [ ]:
from collections import defaultdict
@ray.remote
class LoggingActor(object):
    def __init__(self):
        self.logs = defaultdict(list)
    
    def log(self, index, message):
        self.logs[index].append(message)
    
    def get_logs(self):
        return dict(self.logs)
    
@ray.remote
def run_experiment(experiment_index, logging_actor):
    for i in range(9):
        time.sleep(1)
        # Push a logging message to the actor.
        logging_actor.log.remote(experiment_index, 'On iteration {}'.format(i))    

In [ ]:
# logging_actor = # TODO Instantiate Actor here
experiment_ids = []
for i in range(3):
    # TODO
    # invoke task and append results

In [ ]:
logs = logging_actor.get_logs.remote()
# TODO use ray.get() to fetch the logs

### Next Step
We going to switch a focus little and learn how you can use Ray's replacement
for Python's Multiprocessing pool. 

Let's move on to the [Multiprocessing pool with Ray](ex_05_multiprocess_pool.ipynb)

### Homework

1. Read references below.
2. Can you implement calculating `pi` as a combination of actor (which keeps the state of the progress of calculating `pi` as it approaches its final value) and a task (which  computes candidates for `pi`)? 

**solution hint**: Check the Ray core quickstart docs only if you need to... :-)

---
## References

 * [Writing your First Distributed Python Application with Ray](https://www.anyscale.com/blog/writing-your-first-distributed-python-application-with-ray)
 * [Using and Programming with Actors](https://docs.ray.io/en/latest/actors.html)
 * [Advanced Patterns and Anti-Patterns in Ray](https://docs.ray.io/en/latest/ray-design-patterns/index.htmlhttps://docs.ray.io/en/latest/ray-design-patterns/index.html)

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
➡ [Next notebook](./ex_05_multiprocess_pool.ipynb) <br>
⬅️ [Previous notebook](./ex_03_remote_classes.ipynb) <br>